In [121]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### The Gambler's Ruin

This simulation uses the Optional Stopping Theorem for martingales to solve a classic puzzle: the Gambler's Ruin problem. A gambler with initial capital `c` plays a fair coin toss game. The game ends when they hit a target capital `N` or go broke (0). The theory elegantly predicts the ruin probability as p_ruin = 1 - c/N. This simulation verifies this result by running multiple games and comparing the simulated ruin probability to the theoretical one. The plot shows how the simulated probability converges towards the theoretical value as the number of games increases.

In [122]:
def simulate_gamblers_ruin():
    print("--- Running Simulation: The Gambler's Ruin ---\n")


    initial_capital = 50
    target_capital = 100
    max_simulations = 5000
    prob_win = 0.5

    theoretical_ruin_prob = 1 - (initial_capital / target_capital)

    ruin_count = 0
    simulated_probs = []
    simulation_counts = []

    print(f"Simulating up to {max_simulations} games...")
    print(f"Initial Capital: ${initial_capital}, Target: ${target_capital}")
    print(f"Theoretical Ruin Probability: {theoretical_ruin_prob:.4f}\n")

    for i in range(1, max_simulations + 1):
        capital = initial_capital
        while 0 < capital < target_capital:
            if np.random.rand() < prob_win:
                capital += 1
            else:
                capital -= 1

        if capital == 0:
            ruin_count += 1


        if i % 50 == 0 or i == 1 or i == max_simulations:
            simulated_probs.append(ruin_count / i)
            simulation_counts.append(i)

    final_simulated_prob = ruin_count / max_simulations
    print(f"Simulation Complete.")
    print(f"Final Simulated Ruin Probability: {final_simulated_prob:.4f}")


    fig = go.Figure()


    fig.add_trace(go.Scatter(
        x=simulation_counts,
        y=simulated_probs,
        mode='lines+markers',
        name='Simulated Ruin Probability',
        marker=dict(size=5)
    ))


    fig.add_hline(
        y=theoretical_ruin_prob,
        line_dash="dash",
        line_color="red",
        annotation_text=f"Theoretical Probability ({theoretical_ruin_prob:.2f})",
        annotation_position="bottom right"
    )

    fig.update_layout(
        title_text="Convergence of Gambler's Ruin Simulation",
        xaxis_title="Number of Simulated Games",
        yaxis_title="Probability of Ruin",
        xaxis_type="log",
        yaxis_range=[0,1],
        template="plotly_white"
    )
    fig.show()

In [123]:
simulate_gamblers_ruin()

--- Running Simulation: The Gambler's Ruin ---

Simulating up to 5000 games...
Initial Capital: $50, Target: $100
Theoretical Ruin Probability: 0.5000

Simulation Complete.
Final Simulated Ruin Probability: 0.5120


### De Moivre's Martingale

This simulation explores De Moivre's martingale, a key concept in probability theory defined for a simple symmetric random walk (where each step is +1 or -1 with equal probability). The process $M_n = S_n^2 - n$, where $S_n$ is the position of the random walk at step $n$, is a martingale. This means its expected future value, given the present, is equal to its present value. This simulation visualizes paths of $S_n^2 - n$ and compares them to $S_n^2$, highlighting the driftless nature of the martingale compared to the drifting submartingale $S_n^2$.

In [124]:
def simulate_de_moivres_martingale():
    print("\n--- Running Simulation: De Moivre's Martingale ---\n")

    num_steps = 500
    num_paths = 10

    steps = np.random.choice([-1, 1], size=(num_paths, num_steps))
    S0 = np.zeros((num_paths, 1))
    S = np.cumsum(np.hstack([S0, steps]), axis=1)

    submartingale = S**2
    time_index = np.arange(num_steps + 1)
    martingale = submartingale - time_index

    print("Simulation complete. Visualizing paths...\n")

    fig = go.Figure()
    time_axis = np.arange(num_steps + 1)

    for i in range(min(num_paths, 5)):
        fig.add_trace(go.Scatter(
            x=time_axis, y=martingale[i, :], mode='lines',
            name=f'Martingale Path {i+1}', line=dict(width=2)
        ))

    fig.add_trace(go.Scatter(
        x=time_axis, y=submartingale[0, :], mode='lines',
        name='Submartingale Path (S_n^2)', line=dict(color='rgba(255, 0, 0, 0.6)', dash='dot')
    ))

    fig.add_hline(y=0, line_dash="dash", line_color="black", annotation_text="Fair Game Line (E[M_n]=0)")

    fig.update_layout(
        title="De Moivre's Martingale (S_n^2 - n) vs. Submartingale (S_n^2)",
        xaxis_title="Number of Steps (n)",
        yaxis_title="Value",
        template="plotly_white"
    )
    fig.show()

In [125]:
simulate_de_moivres_martingale()


--- Running Simulation: De Moivre's Martingale ---

Simulation complete. Visualizing paths...



### Bernoulli Exponential Martingale

This simulation explores the Bernoulli Exponential Martingale, another example of a martingale, constructed for a sum of independent Bernoulli random variables. For a sequence of independent Bernoulli(p) trials $\xi_1, \xi_2, \dots$ and $S_n = \sum_{k=1}^n \xi_k$, the process $M_n = \frac{e^{\theta S_n}}{(\phi(\theta))^n}$ is a martingale for any real $\theta$, where $\phi(\theta) = E[e^{\theta \xi_1}]$ is the moment-generating function of a single trial. This simulation visualizes paths of this exponential martingale, demonstrating its property of having a constant expected value (equal to $M_0 = 1$).

In [126]:
def simulate_bernoulli_exponential_martingale():
    print("\n--- Running Simulation: Bernoulli Exponential Martingale ---\n")

    num_steps = 100
    num_paths = 10
    p = 0.4
    theta = 0.5

    phi_theta = p * np.exp(theta * 1) + (1 - p) * np.exp(theta * 0)

    print(f"Parameters: p={p}, theta={theta}")
    print(f"phi(theta) = {phi_theta:.4f}\n")

    trials = np.random.choice([0, 1], size=(num_paths, num_steps), p=[1-p, p])
    S0 = np.zeros((num_paths, 1))
    S = np.cumsum(np.hstack([S0, trials]), axis=1)

    time_index = np.arange(num_steps + 1)
    numerator = np.exp(theta * S)
    denominator = phi_theta ** time_index
    martingale = numerator / denominator

    fig = go.Figure()
    time_axis = np.arange(num_steps + 1)

    for i in range(num_paths):
        fig.add_trace(go.Scatter(
            x=time_axis, y=martingale[i, :], mode='lines',
            name=f'Path {i+1}'
        ))

    fig.add_hline(y=1, line_dash="dash", line_color="red", annotation_text="Expected Value E[M_n] = M_0 = 1")

    fig.update_layout(
        title=f"Exponential Martingale for Bernoulli Trials (p={p}, θ={theta})",
        xaxis_title="Number of Trials (n)",
        yaxis_title="Martingale Value (M_n)",
        yaxis_type="log",
        template="plotly_white"
    )
    fig.show()

In [127]:
simulate_bernoulli_exponential_martingale()


--- Running Simulation: Bernoulli Exponential Martingale ---

Parameters: p=0.4, theta=0.5
phi(theta) = 1.2595



### Pólya's Urn

Pólya's Urn is a classic example of a process that seems like it should have a bias but is perfectly "fair." An urn initially contains `r` red and `b` blue balls. At each step, we draw a ball, note its color, and return it to the urn along with another ball of the same color. The process X_n, defined as the proportion of red balls in the urn after n steps, is a martingale. Its expected future proportion is always its current proportion. This simulation visualizes paths of the proportion of red balls, demonstrating its martingale property.

In [128]:
def simulate_polyas_urn():
    print("\n--- Running Simulation: Pólya's Urn (Martingale) ---\n")

    initial_red = 5
    initial_blue = 5
    num_steps = 200
    num_paths = 10

    all_paths = []
    for _ in range(num_paths):
        red_balls = initial_red
        blue_balls = initial_blue
        proportions = [red_balls / (red_balls + blue_balls)]

        for _ in range(num_steps):
            total_balls = red_balls + blue_balls
            prob_red = red_balls / total_balls

            if np.random.rand() < prob_red:
                red_balls += 1
            else:
                blue_balls += 1

            proportions.append(red_balls / (red_balls + blue_balls))

        all_paths.append(proportions)

    print("Simulation complete. Visualizing paths...\n")

    fig = go.Figure()
    time_axis = np.arange(num_steps + 1)

    for i, path in enumerate(all_paths):
        fig.add_trace(go.Scatter(
            x=time_axis, y=path, mode='lines',
            name=f'Path {i+1}'
        ))

    initial_proportion = initial_red / (initial_red + initial_blue)
    fig.add_hline(
        y=initial_proportion, line_dash="dash", line_color="red",
        annotation_text=f"Initial Proportion / E[X_n] = {initial_proportion:.2f}"
    )

    fig.update_layout(
        title="Pólya's Urn: Proportion of Red Balls (A Martingale)",
        xaxis_title="Number of Draws (n)",
        yaxis_title="Proportion of Red Balls",
        template="plotly_white",
        yaxis=dict(range=[0, 1])
    )
    fig.show()

In [129]:
simulate_polyas_urn()


--- Running Simulation: Pólya's Urn (Martingale) ---

Simulation complete. Visualizing paths...



### Biased Random Walk

A simple symmetric random walk is a martingale. However, if we introduce a bias (e.g., the probability of stepping up is greater than stepping down), the process S_n = sum(X_i) is no longer a "fair game." Since the expected value of each step is positive, the expected future value is always greater than the current value: E[S_{n+1} | F_n] > S_n. This makes it a submartingale, exhibiting a clear upward drift. This simulation visualizes paths of a random walk with a positive bias.

In [130]:
def simulate_biased_random_walk():
    print("\n--- Running Simulation: Biased Random Walk (Submartingale) ---\n")

    num_steps = 300
    num_paths = 10
    p_up = 0.55
    p_down = 1 - p_up

    expected_step = (1 * p_up) + (-1 * p_down)
    print(f"Probability of Up-Step: {p_up:.2f}")
    print(f"Expected drift per step: {expected_step:.2f}\n")

    steps = np.random.choice([1, -1], size=(num_paths, num_steps), p=[p_up, p_down])
    S0 = np.zeros((num_paths, 1))
    S = np.cumsum(np.hstack([S0, steps]), axis=1)

    fig = go.Figure()
    time_axis = np.arange(num_steps + 1)

    for i in range(num_paths):
        fig.add_trace(go.Scatter(x=time_axis, y=S[i, :], mode='lines', name=f'Path {i+1}'))

    drift_line = time_axis * expected_step
    fig.add_trace(go.Scatter(
        x=time_axis, y=drift_line, mode='lines', name='Expected Drift',
        line=dict(color='black', width=4, dash='dash')
    ))

    fig.update_layout(
        title="Biased Random Walk (A Submartingale)",
        xaxis_title="Number of Steps (n)",
        yaxis_title="Value",
        template="plotly_white"
    )
    fig.show()

In [131]:
simulate_biased_random_walk()


--- Running Simulation: Biased Random Walk (Submartingale) ---

Probability of Up-Step: 0.55
Expected drift per step: 0.10



### Wealth in a Sub-Fair Game

This simulates a gambler's wealth in a game with a house edge. The gambler bets a fixed proportion of their current wealth at each step. Since the expected return on each bet is negative, the wealth process W_n is a supermartingale. Its expected future value is always less than its current value: E[W_{n+1} | F_n] < W_n. This process inevitably drifts towards zero, illustrating the concept of "gambler's ruin" in an unfair game. This simulation visualizes the decay of wealth in a sub-fair game.

In [132]:


def simulate_wealth_in_unfair_game():

    print("\n--- Running Simulation: Wealth in a Sub-Fair Game (Supermartingale) ---\n")

    initial_wealth = 100
    bet_fraction = 0.1
    p_win = 0.48
    p_lose = 1 - p_win
    num_steps = 250
    num_paths = 10

    expected_return = p_win - p_lose
    print(f"Probability of Winning: {p_win:.2f}")
    print(f"Expected return per dollar bet: {expected_return:.2f}\n")

    all_paths = []
    for _ in range(num_paths):
        wealth = initial_wealth
        wealth_path = [wealth]
        for _ in range(num_steps):
            if wealth <= 0:
                wealth_path.append(0)
                continue

            bet_amount = wealth * bet_fraction
            if np.random.rand() < p_win:
                wealth += bet_amount
            else:
                wealth -= bet_amount
            wealth_path.append(wealth)
        all_paths.append(wealth_path)

    print("Simulation complete. Visualizing wealth decay...\n")

    fig = go.Figure()
    time_axis = np.arange(num_steps + 1)

    for i, path in enumerate(all_paths):
        fig.add_trace(go.Scatter(x=time_axis, y=path, mode='lines', name=f'Path {i+1}'))

    fig.update_layout(
        title="Gambler's Wealth in an Unfair Game (A Supermartingale)",
        xaxis_title="Number of Bets (n)",
        yaxis_title="Wealth",
        yaxis_type="log",
        template="plotly_white"
    )
    fig.show()


In [133]:
simulate_wealth_in_unfair_game()


--- Running Simulation: Wealth in a Sub-Fair Game (Supermartingale) ---

Probability of Winning: 0.48
Expected return per dollar bet: -0.04

Simulation complete. Visualizing wealth decay...



### Critical Branching Process

A Galton-Watson branching process models a population where each individual in a generation independently produces a random number of offspring for the next generation. If the mean number of offspring per individual (μ) is exactly 1, the process is "critical," and the population size, Z_n, is a martingale. E[Z_{n+1} | F_n] = Z_n * μ = Z_n. These processes almost surely die out, but their expected size remains constant. This simulation visualizes population dynamics in a critical branching process.

In [134]:
def simulate_branching_process():
    print("\n--- Running Simulation: Critical Branching Process (Martingale) ---\n")

    initial_population = 50
    num_generations = 100
    num_paths = 10
    mean_offspring = 1.0
    print(f"Mean offspring (μ) = {mean_offspring:.2f}\n")

    all_paths = []
    for _ in range(num_paths):
        population_path = [initial_population]
        current_population = initial_population
        for _ in range(num_generations):
            if current_population == 0:
                population_path.append(0)
                continue

            offspring = np.random.poisson(mean_offspring, size=current_population)
            current_population = np.sum(offspring)
            population_path.append(current_population)
        all_paths.append(population_path)

    print("Simulation complete. Visualizing population dynamics...\n")

    fig = go.Figure()
    time_axis = np.arange(num_generations + 2)

    for i, path in enumerate(all_paths):
        fig.add_trace(go.Scatter(x=time_axis[:len(path)], y=path, mode='lines', name=f'Path {i+1}'))

    fig.add_hline(
        y=initial_population, line_dash="dash", line_color="green",
        annotation_text=f"Initial Population / E[Z_n] = {initial_population}"
    )

    fig.update_layout(
        title="Critical Branching Process Population (A Martingale)",
        xaxis_title="Generation (n)",
        yaxis_title="Population Size",
        template="plotly_white"
    )
    fig.show()

In [135]:
simulate_branching_process()


--- Running Simulation: Critical Branching Process (Martingale) ---

Mean offspring (μ) = 1.00

Simulation complete. Visualizing population dynamics...



### Absolute Value of a Random Walk

Let S_n be a simple symmetric random walk (a martingale). The process |S_n| is a submartingale. This is a direct consequence of Jensen's inequality for the convex function f(x) = |x|. We have E[|S_{n+1}| | F_n] >= |E[S_{n+1} | F_n]| = |S_n|. The process cannot go below zero and tends to drift upwards away from it. This simulation visualizes the absolute value of a symmetric random walk.

In [136]:
def simulate_absolute_value_random_walk():
    print("\n--- Running Simulation: Absolute Value of Random Walk (Submartingale) ---\n")

    num_steps = 400
    num_paths = 10

    steps = np.random.choice([-1, 1], size=(num_paths, num_steps))
    S0 = np.zeros((num_paths, 1))
    S = np.cumsum(np.hstack([S0, steps]), axis=1)
    submartingale = np.abs(S)

    print("Simulation complete. Visualizing paths...\n")

    fig = go.Figure()
    time_axis = np.arange(num_steps + 1)

    for i in range(num_paths):
        fig.add_trace(go.Scatter(x=time_axis, y=submartingale[i, :], mode='lines', name=f'Path {i+1}'))

    fig.update_layout(
        title="Absolute Value of a Symmetric Random Walk (A Submartingale)",
        xaxis_title="Number of Steps (n)",
        yaxis_title="Absolute Value |S_n|",
        template="plotly_white"
    )
    fig.show()

In [137]:
simulate_absolute_value_random_walk()


--- Running Simulation: Absolute Value of Random Walk (Submartingale) ---

Simulation complete. Visualizing paths...



### Coupon Collector's Problem

In the coupon collector's problem, there are N unique coupons. At each step, we get one coupon chosen uniformly at random. Let X_n be the number of *unique* coupons yet to be collected after n draws. This process is a supermartingale. The probability of finding a new coupon at the next step is X_n / N. So, E[X_{n+1} | F_n] | X_n. The process has a downward drift and will eventually reach 0. This simulation visualizes the number of remaining coupons to collect.

In [138]:
def simulate_coupon_collector():
    print("\n--- Running Simulation: Coupon Collector's Problem (Supermartingale) ---\n")

    num_unique_coupons = 50
    max_draws = 300
    num_paths = 10
    print(f"Total unique coupons to collect: {num_unique_coupons}\n")

    all_paths = []
    for _ in range(num_paths):
        collected_coupons = set()
        remaining_path = [num_unique_coupons]
        for _ in range(max_draws):
            draw = np.random.randint(1, num_unique_coupons + 1)
            collected_coupons.add(draw)
            remaining_path.append(num_unique_coupons - len(collected_coupons))
            if len(collected_coupons) == num_unique_coupons:
                break
        all_paths.append(remaining_path)

    print("Simulation complete. Visualizing decay...\n")

    fig = go.Figure()
    time_axis = np.arange(max_draws + 2)

    for i, path in enumerate(all_paths):
        fig.add_trace(go.Scatter(x=time_axis[:len(path)], y=path, mode='lines', name=f'Path {i+1}'))

    fig.update_layout(
        title="Coupon Collector's Problem: Remaining Coupons (A Supermartingale)",
        xaxis_title="Number of Draws (n)",
        yaxis_title="Number of Unique Coupons Remaining",
        template="plotly_white"
    )
    fig.show()

In [139]:
simulate_coupon_collector()


--- Running Simulation: Coupon Collector's Problem (Supermartingale) ---

Total unique coupons to collect: 50

Simulation complete. Visualizing decay...



### The Likelihood Ratio

A fundamental martingale in statistics and information theory. Imagine we are observing a sequence of coin flips, but we don't know if the coin is fair (P) or biased (Q). The likelihood ratio L_n is the relative probability of observing the sequence so far under Q versus P. If the flips are truly generated by the fair coin (P), then the process L_n is a martingale. This process is the foundation of the sequential probability ratio test. This simulation visualizes the likelihood ratio process under the null hypothesis.

In [140]:
def simulate_likelihood_ratio():
    print("\n--- Running Simulation: Likelihood Ratio (Martingale) ---\n")

    p0 = 0.5
    p1 = 0.6
    num_flips = 200
    num_paths = 10

    print(f"True Probability P(H) = {p0}, Alternative Q(H) = {p1}\n")

    all_paths = []
    for _ in range(num_paths):
        flips = np.random.choice([1, 0], size=num_flips, p=[p0, 1-p0])

        likelihood_ratio = 1.0
        path = [likelihood_ratio]

        for flip in flips:
            if flip == 1:
                ratio_of_probs = p1 / p0
            else:
                ratio_of_probs = (1 - p1) / (1 - p0)

            likelihood_ratio *= ratio_of_probs
            path.append(likelihood_ratio)
        all_paths.append(path)

    fig = go.Figure()
    time_axis = np.arange(num_flips + 1)

    for i, path in enumerate(all_paths):
        fig.add_trace(go.Scatter(x=time_axis, y=path, mode='lines', name=f'Path {i+1}'))

    fig.add_hline(y=1, line_dash="dash", line_color="black", annotation_text="E[L_n] = L_0 = 1")

    fig.update_layout(
        title="Likelihood Ratio Process (A Martingale under P)",
        xaxis_title="Number of Coin Flips (n)",
        yaxis_title="Likelihood Ratio L_n",
        yaxis_type="log",
        template="plotly_white"
    )
    fig.show()

In [141]:
simulate_likelihood_ratio()


--- Running Simulation: Likelihood Ratio (Martingale) ---

True Probability P(H) = 0.5, Alternative Q(H) = 0.6



### Maximum of a Random Walk

Let S_n be a simple symmetric random walk (a martingale). The process M_n, defined as the maximum value the walk has achieved up to time n (M_n = max(S_0, S_1, ..., S_n)), is a submartingale. Intuitively, the maximum can only increase or stay the same at the next step, so E[M_{n+1} | F_n] >= M_n. It has a clear upward drift as the random walk explores new highs. This simulation visualizes the running maximum of a symmetric random walk.

In [142]:
def simulate_rw_maximum():
    print("\n--- Running Simulation: Maximum of a Random Walk (Submartingale) ---\n")

    num_steps = 500
    num_paths = 10

    steps = np.random.choice([-1, 1], size=(num_paths, num_steps))
    S0 = np.zeros((num_paths, 1))
    S = np.cumsum(np.hstack([S0, steps]), axis=1)

    submartingale = np.maximum.accumulate(S, axis=1)

    print("Simulation complete. Visualizing paths...\n")

    fig = go.Figure()
    time_axis = np.arange(num_steps + 1)

    fig.add_trace(go.Scatter(
        x=time_axis, y=S[0, :], mode='lines', name='Random Walk (S_n)',
        line=dict(color='grey', dash='dash')
    ))

    fig.add_trace(go.Scatter(
        x=time_axis, y=submartingale[0, :], mode='lines',
        name='Maximum Process (M_n)', line=dict(color='crimson', width=3)
    ))

    for i in range(1, 5):
        fig.add_trace(go.Scatter(
            x=time_axis, y=submartingale[i, :], mode='lines',
            name=f'M_n Path {i+1}', line=dict(width=1.5, color='lightblue'),
            showlegend=False
        ))

    fig.update_layout(
        title="Running Maximum of a Symmetric Random Walk (A Submartingale)",
        xaxis_title="Number of Steps (n)",
        yaxis_title="Value",
        template="plotly_white"
    )
    fig.show()

In [143]:
simulate_rw_maximum()


--- Running Simulation: Maximum of a Random Walk (Submartingale) ---

Simulation complete. Visualizing paths...



### Fair Game with a Cost

This simulates a gambler's wealth in a game that would be fair, but has a small, fixed transaction cost for every bet placed. For example, a coin flip where you win or lose $1, but must pay $0.05 to play each round. The expected change in wealth at each step is negative (equal to -cost), so the wealth process W_n is a supermartingale. Its expected future value is always less than its current value: E[W_{n+1} | F_n] < W_n. This process demonstrates how even a tiny "house edge" or cost creates a clear downward drift over time. This simulation visualizes wealth in a fair game with a transaction cost.

In [144]:
def simulate_fair_game_with_cost():
    print("\n--- Running Simulation: Fair Game with a Cost (Supermartingale) ---\n")

    initial_wealth = 20
    num_bets = 400
    num_paths = 10
    cost_per_bet = 0.05

    print(f"Cost per bet = {cost_per_bet}\n")

    bet_outcomes = np.random.choice([-1, 1], size=(num_paths, num_bets))
    wealth_changes = bet_outcomes - cost_per_bet

    W0 = np.full((num_paths, 1), initial_wealth)
    W = np.cumsum(np.hstack([W0, wealth_changes]), axis=1)

    fig = go.Figure()
    time_axis = np.arange(num_bets + 1)

    for i in range(num_paths):
        fig.add_trace(go.Scatter(x=time_axis, y=W[i,:], mode='lines', name=f'Path {i+1}'))

    decay_line = initial_wealth - (time_axis * cost_per_bet)
    fig.add_trace(go.Scatter(
        x=time_axis, y=decay_line, mode='lines', name='Expected Wealth',
        line=dict(color='black', width=4, dash='dash')
    ))

    fig.update_layout(
        title="Wealth in a Fair Game with a Transaction Cost (A Supermartingale)",
        xaxis_title="Number of Bets (n)",
        yaxis_title="Wealth",
        template="plotly_white"
    )
    fig.show()

In [145]:
simulate_fair_game_with_cost()


--- Running Simulation: Fair Game with a Cost (Supermartingale) ---

Cost per bet = 0.05

